In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
%matplotlib inline

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset

train = pd.read_csv('../data/sensor/train.csv')
test = pd.read_csv('../data/sensor/test.csv')
train_labels = pd.read_csv("../data/sensor/train_labels.csv")

train = train.set_index(["sequence", "subject", "step"])
test = test.set_index(["sequence", "subject", "step"])

In [7]:
features = ["sensor_{:02d}".format(i) for i in range(13)]
input_size = train.shape[1]
sequence_length = len(train.index.get_level_values(2).unique())

# Scaling test and train
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

# Reshaping:
train = train.reshape(-1, sequence_length, input_size)
test = test.reshape(-1, sequence_length, input_size)
print("After Reshape")
print("Shape of training set: {}".format(train.shape))
print("Shape of test set: {}".format(test.shape))

After Reshape
Shape of training set: (25968, 60, 13)
Shape of test set: (12218, 60, 13)


In [8]:
# Splitting train data set into train and validation sets
# validation size is selected as 0.2
t_X, v_X, t_y, v_y = train_test_split(train, train_labels.state, test_size=0.20,
                                      shuffle=True, random_state=0)

# Converting train, validation and test data into tensors
train_X_tensor = torch.tensor(t_X).float()
val_X_tensor = torch.tensor(v_X).float()
test_tensor = torch.tensor(test).float()

# Converting train and validation labels into tensors
train_y_tensor = torch.tensor(t_y.values)
val_y_tensor = torch.tensor(v_y.values)

# Creating train and validation tensors
train_tensor = TensorDataset(train_X_tensor, train_y_tensor)
val_tensor = TensorDataset(val_X_tensor, val_y_tensor)

# Defining the dataloaders
dataloaders = dict()
dataloaders["train"] = DataLoader(train_tensor, batch_size=64, shuffle=True)
dataloaders["val"] = DataLoader(val_tensor, batch_size=32)
dataloaders["test"] = DataLoader(test_tensor, batch_size=32)
print("Dataloaders are created!")

Dataloaders are created!


In [9]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, seq_len, dropout=0.5, output_size=1):
        super(RNN, self).__init__()
        
        # LSTM Layers
        self.lstm_1 = nn.LSTM(input_size, hidden_sizes[0], num_layers=2,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_21 = nn.LSTM(2*hidden_sizes[0], hidden_sizes[1], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_22 = nn.LSTM(input_size, hidden_sizes[1], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_31 = nn.LSTM(2*hidden_sizes[1], hidden_sizes[2], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_32 = nn.LSTM(4*hidden_sizes[1], hidden_sizes[2], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_41 = nn.LSTM(2*hidden_sizes[2], hidden_sizes[3], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_42 = nn.LSTM(4*hidden_sizes[2], hidden_sizes[3], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        hidd = 2*hidden_sizes[0] + 4*(hidden_sizes[1]+hidden_sizes[2]+hidden_sizes[3])
        self.lstm_5 = nn.LSTM(hidd, hidden_sizes[4], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        
        # Fully Connected Layer
        self.fc = nn.Sequential(nn.Linear(2*hidden_sizes[4]*seq_len, 4096),
                                nn.ReLU(inplace=True),
                                nn.Dropout(p=dropout),
                                nn.Linear(4096, 1024),
                                nn.ReLU(inplace=True),
                                nn.Dropout(p=dropout),
                                nn.Linear(1024, output_size),
                                nn.Sigmoid()
                               )
        
    def forward(self, x):
        # lstm layers:
        x1, _ = self.lstm_1(x)
        
        x_x1, _ = self.lstm_21(x1)
        x_x2, _ = self.lstm_22(x)
        x2 = torch.cat([x_x1, x_x2], dim=2)
        
        x_x1, _ = self.lstm_31(x_x1)
        x_x2, _ = self.lstm_32(x2)
        x3 = torch.cat([x_x1, x_x2], dim=2)
        
        x_x1, _ = self.lstm_41(x_x1)
        x_x2, _ = self.lstm_42(x3)
        x4 = torch.cat([x_x1, x_x2], dim=2)
        x = torch.cat([x1, x2, x3, x4], dim=2)
        x, _ = self.lstm_5(x)
        
        # fully connected layers:
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

In [11]:
hidden_sizes = [288, 192, 144, 96, 32]
max_learning_rate = 0.001
epochs = 41
model = RNN(input_size, hidden_sizes, sequence_length)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=max_learning_rate)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_losses = []
val_losses = []
for epoch in range(epochs):
    train_loss = 0
    val_loss = 0
    model.train()
    for inputs, labels in dataloaders["train"]:
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.unsqueeze(1).float()

        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_losses.append(train_loss/len(dataloaders["train"]))
    
    model.eval()
    with torch.no_grad():
        for inputs, labels in dataloaders["val"]:
            inputs, labels = inputs.to(device), labels.to(device)
            output = model(inputs)
            loss = criterion(output.squeeze(), labels.float())
            val_loss += loss.item()
        val_losses.append(val_loss/len(dataloaders["val"]))
    
    print("Epoch: {}/{}.. ".format(epoch+1, epochs),
          "Training Loss: {:.3f}.. ".format(train_losses[-1]),
          "Validation Loss: {:.3f}.. ".format(val_losses[-1]))


Epoch: 1/41..  Training Loss: 0.690..  Validation Loss: 0.680.. 
Epoch: 2/41..  Training Loss: 0.680..  Validation Loss: 0.686.. 
Epoch: 3/41..  Training Loss: 0.679..  Validation Loss: 0.681.. 


KeyboardInterrupt: 